In [13]:
import requests
import pymysql
from datetime import datetime
import logging
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set up logging
logging.basicConfig(level=logging.INFO)

# MySQL database connection settings
db_config = {
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': int(os.getenv('DB_PORT')),
    'database': os.getenv('DB_NAME'),
    #Comment this out when running on local machine
    'ssl': {'ca': r'ca-certificate.crt'}
  
}



In [14]:
class DatabaseManager:
    """Class to manage database connections and operations."""
    
    def __init__(self, config):
        self.config = config
        self.connection = self.establish_connection()

    def establish_connection(self):
        """Establish a connection to the MySQL database."""
        try:
            connection = pymysql.connect(
                host=self.config['host'],
                user=self.config['user'],
                password=self.config['password'],
                database=self.config['database'],
                port=self.config['port'],
                #Comment this out when running on local machine
                ssl=self.config['ssl']
            )
            logging.info("Connected to MySQL database")
            return connection
        except pymysql.MySQLError as err:
            logging.error(f"Database connection error: {err}")
            return None
        
    def run_query(self, query):
        """Run a query on the database."""
        try:
            with self.connection.cursor() as cursor:
                cursor.execute(query)
                self.connection.commit()
                logging.info("Query executed successfully")
                #output the result of the query
                return cursor.fetchall()
        except pymysql.MySQLError as err:
            logging.error(f"Query execution error: {err}")
            return None


In [15]:
db_manager = DatabaseManager(db_config)

INFO:root:Connected to MySQL database


In [16]:
#run a query to show tables
query = "SHOW TABLES"
db_manager.run_query(query)

INFO:root:Query executed successfully


(('environment',), ('tomtom',))

In [17]:
#query to count all the rows in the table
query = "SELECT COUNT(*) FROM tomtom"
db_manager.run_query(query)

INFO:root:Query executed successfully


((3248,),)

In [18]:
import pandas as pd

#query to get all the data from the table
query = "SELECT * FROM tomtom"
data = db_manager.run_query(query)

#convert the data to a pandas dataframe
df_tomtom = pd.DataFrame(data)

#show the first 5 rows of the dataframe

print(df_tomtom.head())

query_env = "SELECT * FROM environment"
data_env = db_manager.run_query(query_env)
df_env = pd.DataFrame(data_env)
print(df_env.head())



INFO:root:Query executed successfully
INFO:root:Query executed successfully


   0           1     2     3     4     5     6     7     8     9     10
0   1  1728369760  49.0  49.0  51.0  51.0  49.0  49.0  51.0  51.0  53.0
1   2  1728370802  49.0  49.0  40.0  40.0  49.0  49.0  40.0  40.0  53.0
2   3  1728374402  39.0  39.0  37.0  37.0  39.0  39.0  37.0  37.0  45.0
3   4  1728378001  49.0  49.0  39.0  39.0  49.0  49.0  39.0  39.0  53.0
4   5  1728381601  49.0  49.0  39.0  39.0  49.0  49.0  39.0  39.0  53.0
   0           1                        2     3      4              5     6  \
0  1  1728369760  Blanchardstown, Ireland  27.0  12.44     light rain  2.30   
1  2  1728370802  Blanchardstown, Ireland  27.0  12.47  broken clouds  0.54   
2  3  1728374402  Blanchardstown, Ireland  27.0  12.84  broken clouds  2.13   
3  4  1728378001  Blanchardstown, Ireland  26.0  13.18  broken clouds  3.77   
4  5  1728381601  Blanchardstown, Ireland  25.0  13.57  broken clouds  3.90   

      7  
0  0.24  
1  0.00  
2  0.00  
3  0.00  
4  0.00  
